Data sources from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
Excercise in producing a decision tree to predict forest cover of parcels of land based on the 50+ tributes.
Based on the case study in Advnaced Analytics using Spark
Using Almond Shell kernel for Jupyter notebook

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Logger,Level}

import org.apache.log4j.{Logger,Level}

In [3]:
Logger.getLogger("org").setLevel(Level.ERROR)

Create the sql sessions 

In [4]:
val spark = {
            org.apache.spark.sql.NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@6ed3712

In [5]:
val data = spark.read
            .option("header","false")
            .option("inferSchema","true")
            .csv("../covtype/")

csv at cmd4.sc:4

1 / 1

csv at cmd4.sc:4

4 / 4

data: DataFrame = [_c0: string, _c1: string ... 53 more fields]

In [6]:
data.count()

count at cmd5.sc:1

4 / 4

count at cmd5.sc:1

1 / 1

res5: Long = 581259L

In [7]:
data.head()

head at cmd6.sc:1

1 / 1

res6: Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5]

In [8]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _c27

Looks like the schema isn't what was expected. _c0..2 should be integers

In [9]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._

Have to update the scheme. Only way seems to be to create new DF every time.

In [10]:
val data2=data.withColumn("_c0",data("_c0").cast(IntegerType))
val data3=data2.withColumn("_c1",data2("_c1").cast(IntegerType))
val data4=data3.withColumn("_c2",data3("_c2").cast(IntegerType))

data2: DataFrame = [_c0: int, _c1: string ... 53 more fields]
data3: DataFrame = [_c0: int, _c1: int ... 53 more fields]
data4: DataFrame = [_c0: int, _c1: int ... 53 more fields]

In [11]:
data4.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

Above approach worked by duplicating the dataframe everytime seem thereis potential for memory overflow. Try doing it when adding titles.

Create header titles for the columns. 

In [12]:
val colNames = Seq(
"Elevation", "Aspect", "Slope",
"Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"
) ++ (
(0 until 4).map(i => s"Wilderness_Area_$i")
) ++ (
(0 until 40).map(i => s"Soil_Type_$i")
) ++ Seq("Cover_Type")

colNames: Seq[String] = List(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...

In [13]:
colNames.getClass

res12: Class[T] = class scala.collection.immutable.$colon$colon

implicits contains the function $ which returns a column object. Needed to access the data type in order to cast it to integer. $("string"): column   ... pretty 'hacky' syntax.

In [14]:
import spark.implicits._

import spark.implicits._

In [15]:
val dataClean = data.toDF(colNames:_*).
    withColumn("Elevation",$"Elevation".cast(IntegerType))
    .withColumn("Aspect",$"Aspect".cast(IntegerType))
    .withColumn("Slope",$"Slope".cast(IntegerType))
    .withColumn("Cover_Type",$"Cover_Type".cast(DoubleType))


dataClean: DataFrame = [Elevation: int, Aspect: int ... 53 more fields]

In [16]:
dataClean.show() // Using the spark.sql API

show at cmd15.sc:1

1 / 1

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness

|     2617|    45|    9|                             240|                            56|                            666|          223|           221|          133|                              6244|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2612|    59|   10|                             247|                            11|                            636|          228|           219|          124|                              6230|          

|     2504|     0|    4|                              95|                             5|                            691|          214|           232|          156|                              5572|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2503|    38|    5|                              85|                            10|                            741|          220|           228|          144|                              5555|          

Randomly split the data into train and test using 10% for testing (will need to further split the training data if CV performed

In [17]:
val Array(trainData, testData) = dataClean.randomSplit(Array(0.9,0.1))
trainData.cache()
testData.cache()

trainData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res16_1: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res16_2: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]

Vectorise the features. This is required by all SparkML alogorithms.

Took some messing around to find the right repo for ml lib. Eventually found a reference on the Mvnrepository site to find the mllib version that goes with scala 2.12.

In [18]:
import $ivy.`org.apache.spark::spark-mllib:2.4.0`

import $ivy.$                                    

In [19]:
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.feature.VectorAssembler

In [20]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new org.apache.spark.ml.feature.VectorAssembler()
                    .setInputCols(inputCols)
                    .setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_d994a0f5f03d

Since spark 2.4 vector assembler extends HadHandleInvalid which requires you to specify how nulls will be handled. It take a single parameter ("skip") to skip when encountered. If not used then the dataset must be cleaned up and the missing data imputed. 

See stackoverflow
https://stackoverflow.com/questions/41362295/sparkexception-values-to-assemble-cannot-be-null


TODO: Investigate the source of nulls

In [21]:
val assembledTrainData = assembler.setHandleInvalid("skip").transform(trainData)

assembledTrainData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

In [22]:
assembledTrainData.select("featureVector").show(truncate=false)

show at cmd21.sc:1

1 / 1

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,14],[1889.0,28.0,22.0,150.0,23.0,120.0,205.0,185.0,108.0,759.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1898.0,34.0,23.0,175.0,56.0,134.0,210.0,184.0,99.0,765.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1899.0,355.0,22.0,153.0,43.0,1

In [23]:
assembledTrainData.getClass

res22: Class[T] = class org.apache.spark.sql.Dataset

In [24]:
trainData.getClass

res23: Class[T] = class org.apache.spark.sql.Dataset

Note on above. The transform fucntion converts a dataframe to a dataframe but re-organises it.

Now to create the DecisionTreem classifier for predicting the group cover type (a categorical valiable ranging 1..7)

In [25]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random

import org.apache.spark.ml.classification.DecisionTreeClassifier

import scala.util.Random

In [26]:
val classifier = new org.apache.spark.ml.classification.DecisionTreeClassifier()
                        .setSeed(Random.nextLong())
                        .setLabelCol("Cover_Type")
                        .setFeaturesCol("featureVector")
                        .setPredictionCol("prediction")


classifier: DecisionTreeClassifier = dtc_e1f4933163ee

In [27]:
val model = classifier.fit(assembledTrainData)
//print(model.toDebugString)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

4 / 4

collectAsMap at RandomForest.scala:557

4 / 4

model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_e1f4933163ee) of depth 5 with 41 nodes

In [28]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_e1f4933163ee) of depth 5 with 41 nodes
  If (feature 0 <= 3048.5)
   If (feature 0 <= 2555.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2441.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2441.5)
      Predict: 3.0
    Else (feature 10 > 0.5)
     Predict: 2.0
   Else (feature 0 > 2555.5)
    If (feature 0 <= 2953.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 0 > 2953.5)
     If (feature 3 <= 211.0)
      If (feature 36 <= 0.5)
       Predict: 2.0
      Else (feature 36 > 0.5)
       Predict: 1.0
     Else (feature 3 > 211.0)
      Predict: 2.0
  Else (feature 0 > 3048.5)
   If (feature 0 <= 3329.5)
    If (feature 7 <= 239.5)
     Predict: 1.0
    Else (feature 7 > 239.5)
     If (feature 3 <= 340.5)
      Predict

Cool! Above cell appears to show a recursive definition (and almost self explaining) of the decision tree model.
So now the model has been fit (estimator(ed) in spark speak) we can use the model to make predictions on the test data. 
Note that the default settings have been used on the above model so we don't expect this out of the box model to do very well.

Without getting under the hood I assume that the decison tree optimises the gini index for the tree. TODO find out.

DecisiontTreeClassifiers have an 'importance' dunction to assess the importance of each feature. Not obvious what this is but assume its some sort of sensitivity analysis.??

In [29]:
model.featureImportances.toArray.zip(inputCols).sorted.reverse.foreach(println)

(0.8113824140909414,Elevation)
(0.0385123310223562,Horizontal_Distance_To_Hydrology)
(0.029176618274391993,Wilderness_Area_0)
(0.02716972082184828,Hillshade_Noon)
(0.02567498610101623,Soil_Type_1)
(0.02302326455300907,Soil_Type_3)
(0.01820147233342822,Soil_Type_31)
(0.011446099378485014,Wilderness_Area_2)
(0.0063126286694189715,Soil_Type_22)
(0.003495266174309226,Horizontal_Distance_To_Fire_Points)
(0.0033945286035579145,Vertical_Distance_To_Hydrology)
(0.002210669977237501,Horizontal_Distance_To_Roadways)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Soil_Type_9)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soil_Type_21)
(0.0,Soil

So the above model has the elevation as by far and away the most important feature. However we have not assessed the performance of modey yet so this should be treated cautiously until this is done. 

Note that the decisionTreeClassificationModel is a transformed ( in spark speak this is just a function that transform a dataframe into a dataframe

In [30]:
val preds = model.transform(assembledTrainData)

preds.select("Cover_Type","prediction","probability").show(truncate=false)

show at cmd29.sc:3

1 / 1

+----------+----------+--------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                       |
+----------+----------+--------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.0,0.030580492864551664,0.6356450516828213,0.052255087807146175,0.0,0.28151936764548086,0.0]|
|6.0       |4.0       |[0.0,0.0,0.03940217391304348,0.29528985507246375,0.4433876811594203,0.0,0.22192028985507245,0.0]  |
|6.0       |3.0       |[0.0,0.0,0.030580492864551664,0.6356450516828213,0.052255087807146175,0.0,0.28151936764548086,0.0]|
|6.0       |3.0       |[0.0,0.0,0.030580492864551664,0.6356450516828213,0.052255087807146175,0.0,0.28151936764548086,0.0]|
|6.0       |3.0       |[0.0,0.0,0.030580492864551664,0.6356450516828213,0.052255087807146175,0.0,0.28151936764548086,0.0]|
|6.0       |3.0 

preds: DataFrame = [Elevation: int, Aspect: int ... 57 more fields]

In [31]:
preds.columns.foreach(println)

Elevation
Aspect
Slope
Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Hillshade_9am
Hillshade_Noon
Hillshade_3pm
Horizontal_Distance_To_Fire_Points
Wilderness_Area_0
Wilderness_Area_1
Wilderness_Area_2
Wilderness_Area_3
Soil_Type_0
Soil_Type_1
Soil_Type_2
Soil_Type_3
Soil_Type_4
Soil_Type_5
Soil_Type_6
Soil_Type_7
Soil_Type_8
Soil_Type_9
Soil_Type_10
Soil_Type_11
Soil_Type_12
Soil_Type_13
Soil_Type_14
Soil_Type_15
Soil_Type_16
Soil_Type_17
Soil_Type_18
Soil_Type_19
Soil_Type_20
Soil_Type_21
Soil_Type_22
Soil_Type_23
Soil_Type_24
Soil_Type_25
Soil_Type_26
Soil_Type_27
Soil_Type_28
Soil_Type_29
Soil_Type_30
Soil_Type_31
Soil_Type_32
Soil_Type_33
Soil_Type_34
Soil_Type_35
Soil_Type_36
Soil_Type_37
Soil_Type_38
Soil_Type_39
Cover_Type
featureVector
rawPrediction
probability
prediction


In [32]:
val results = preds.select("Cover_Type","prediction")

results: DataFrame = [Cover_Type: double, prediction: double]

In [33]:
results.selectExpr("Cover_Type==prediction as Matched","Cover_Type")
    .groupBy("Cover_Type").pivot("Matched",Seq("true","false")).count().show()

show at cmd32.sc:2

4 / 4

show at cmd32.sc:2

200 / 200

show at cmd32.sc:2

...

show at cmd32.sc:2

100 / 100

show at cmd32.sc:2

75 / 75

+----------+------+-----+
|Cover_Type|  true|false|
+----------+------+-----+
|       7.0|  9598| 8785|
|       1.0|134899|55717|
|       4.0|   979| 1488|
|       3.0| 29234| 2984|
|       2.0|192125|62975|
|       6.0|  null|15598|
|       5.0|  null| 8531|
+----------+------+-----+



In [34]:
val predPivot = results.selectExpr("Cover_Type","prediction")
    .groupBy("Cover_Type")
    .pivot("prediction",(1 to 7))
    .count()
    .na.fill(0).sort("Cover_Type")

predPivot: Dataset[Row] = [Cover_Type: double, 1: bigint ... 6 more fields]

In [35]:
predPivot.show()

show at cmd34.sc:1

4 / 4

show at cmd34.sc:1

200 / 200

show at cmd34.sc:1

200 / 200

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|134899| 51513|  237|  0|  0|  0|3967|
|       2.0| 54434|192125| 7991| 87|  0|  0| 463|
|       3.0|     0|  2332|29234|652|  0|  0|   0|
|       4.0|     0|     0| 1488|979|  0|  0|   0|
|       5.0|    10|  7743|  778|  0|  0|  0|   0|
|       6.0|     0|  3088|12020|490|  0|  0|   0|
|       7.0|  8715|    16|   54|  0|  0|  0|9598|
+----------+------+------+-----+---+---+---+----+



TODO: Add row total

In [36]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

In [37]:
val evaluator = new org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator()
                .setLabelCol("Cover_Type")
                .setPredictionCol("prediction")

evaluator: MulticlassClassificationEvaluator = mcEval_e714437e7f00

In [38]:
evaluator.setMetricName("accuracy").evaluate(preds)

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res37: Double = 0.7015220505131828

As expected accuracy not great. Also, looking at the prediction pivot table shows that there is class imbalance that might be affecting the utility of accuracy and f1 might be a better score.


In [39]:
evaluator.setMetricName("f1").evaluate(preds)

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

map at MulticlassMetrics.scala:45

...

map at MulticlassMetrics.scala:50

4 / 4

collectAsMap at MulticlassMetrics.scala:52

4 / 4

res38: Double = 0.6848892553807712

Actually not that difference suggesting that the algorithm (with the current set of parameters) isn't greatly affected by the class imbalance. This should be monitored on going as the imbalance of the data warrants this.

Its seems that spark has a confusion matrix ( what I produced in the above pivot table).Its accesses (in the book at least) via the Mllib rdd API (which is in maintenance mode now with the dataframe spark.ml api being the promoted one)

In [40]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics
val predictionRDD = preds.
        select("prediction","Cover_Type").
        as[(Double,Double)].
        rdd
val multiclassMetrics = new org.apache.spark.mllib.evaluation.MulticlassMetrics(predictionRDD)

import org.apache.spark.mllib.evaluation.MulticlassMetrics

predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[132] at rdd at cmd39.sc:4
multiclassMetrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@3d9a298d

In [41]:
multiclassMetrics.confusionMatrix

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

map at MulticlassMetrics.scala:55

4 / 4

collectAsMap at MulticlassMetrics.scala:57

4 / 4

res40: org.apache.spark.mllib.linalg.Matrix = 134899.0  51513.0   237.0    0.0    0.0  0.0  3967.0  
54434.0   192125.0  7991.0   87.0   0.0  0.0  463.0   
0.0       2332.0    29234.0  652.0  0.0  0.0  0.0     
0.0       0.0       1488.0   979.0  0.0  0.0  0.0     
10.0      7743.0    778.0    0.0    0.0  0.0  0.0     
0.0       3088.0    12020.0  490.0  0.0  0.0  0.0     
8715.0    16.0      54.0     0.0    0.0  0.0  9598.0  

In [42]:
predPivot.show()

show at cmd41.sc:1

4 / 4

show at cmd41.sc:1

200 / 200

show at cmd41.sc:1

200 / 200

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|134899| 51513|  237|  0|  0|  0|3967|
|       2.0| 54434|192125| 7991| 87|  0|  0| 463|
|       3.0|     0|  2332|29234|652|  0|  0|   0|
|       4.0|     0|     0| 1488|979|  0|  0|   0|
|       5.0|    10|  7743|  778|  0|  0|  0|   0|
|       6.0|     0|  3088|12020|490|  0|  0|   0|
|       7.0|  8715|    16|   54|  0|  0|  0|9598|
+----------+------+------+-----+---+---+---+----+



To evaluate the 70% accuracy from the default model we can benchmark it against a null model and use an inner product of the class probabilities to compare.

In [43]:
def classProbs(df: DataFrame):Array[Double] = {
    val totalCounts = df.count().asInstanceOf[Double]
        df.groupBy("Cover_Type").count()
        .orderBy("Cover_Type")
        .select("count").as[Double]
        .map(_/totalCounts)
        .collect()
}

defined function classProbs

Take inner product of test and train class probabilities as A measure of similarity

In [44]:
val trainProbs = classProbs(trainData)
val testProbs = classProbs(testData)
trainProbs.zip(testProbs).map(x=>x._1*x._2).sum

count at cmd42.sc:2

4 / 4

count at cmd42.sc:2

1 / 1

collect at cmd42.sc:6

...

collect at cmd42.sc:6

200 / 200

collect at cmd42.sc:6

8 / 8

trainProbs: Array[Double] = Array(
  4.2818611568288997E-4,
  0.36437109208486496,
  0.4876351701370769,
  0.06158616194228281,
  0.004715781907989686,
  0.016307391753976493,
  0.02981628139473981,
  0.035139934663386455
)
testProbs: Array[Double] = Array(
  3.9571934895564503E-4,
  0.3651629331406352,
  0.4852035373868759,
  0.06083754860465917,
  0.004817452943807852,
  0.01655139189979698,
  0.030435979491414612,
  0.03659543718385465
)
res43_2: Double = 0.3758901238940733

The null model of randomly choosing a cover type label and using it as the prediction yields a 37% true positive rate. Makes the 70% TP rate look pretty good. 